In [2]:
!wget https://strogino.duckdns.org/binary_Xception_old.h5
!wget https://strogino.duckdns.org/multi_label.old.h5
!wget https://strogino.duckdns.org/segmentation_defect_1.h5

--2024-03-08 20:38:58--  https://strogino.duckdns.org/binary_Xception_old.h5
Resolving strogino.duckdns.org (strogino.duckdns.org)... 46.188.28.121
Connecting to strogino.duckdns.org (strogino.duckdns.org)|46.188.28.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115833648 (110M) [application/octet-stream]
Saving to: ‘binary_Xception_old.h5’

binary_Xception_old 100%[===================>] 110.47M  5.78MB/s    in 13s     

2024-03-08 20:39:12 (8.52 MB/s) - ‘binary_Xception_old.h5’ saved [115833648/115833648]

--2024-03-08 20:39:12--  https://strogino.duckdns.org/multi_label.old.h5
Resolving strogino.duckdns.org (strogino.duckdns.org)... 46.188.28.121
Connecting to strogino.duckdns.org (strogino.duckdns.org)|46.188.28.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115824536 (110M) [application/octet-stream]
Saving to: ‘multi_label.old.h5’

multi_label.old.h5  100%[===================>] 110.46M  9.65MB/s    in 11s     

2024-03-

In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install kaggle
!mkdir -p ~/.kaggle
!rm -rf /content/steel/
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c severstal-steel-defect-detection
!7z x -bsp2 -bd severstal-steel-defect-detection.zip -o/content/steel
contents = os.listdir('/content/steel/')

100% 1.57G/1.57G [00:20<00:00, 57.5MB/s]
100% 1.57G/1.57G [00:20<00:00, 81.4MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1684204253 bytes (1607 MiB)

Extracting archive: severstal-steel-defect-detection.zip
 22% 4096 Open              --
Path = severstal-steel-defect-detection.zip
Type = zip
Physical Size = 1684204253

  0%      0% 102 - test_images/04e73b12f.jpg                                      1% 209 - test_images/097311f2c.jpg                                      1% 316 - test_images/0e6b42f13.jpg                                    

In [5]:
!pip install tensorflow==2.8.0 keras==2.8.0
!pip install segmentation-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing 

In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle
import matplotlib.patches as patches
import re
import random
from sklearn.model_selection import train_test_split
import cv2
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from PIL import Image
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model,load_model
from keras.regularizers import l2
import datetime
%load_ext tensorboard
import segmentation_models
from segmentation_models import Unet
from segmentation_models import get_preprocessing
import imgaug.augmenters as iaa
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()
from tensorflow.keras.losses import binary_crossentropy
from tqdm import tqdm

Segmentation Models: using `keras` framework.


In [7]:
test_image=[i for i in os.listdir('/content/steel/test_images')]

In [8]:
len(test_image)

5506

## Final Pipeline



    Binary Classifier trained with all Images with 1 output probability i.e. [defect] which tells whether whether Image have defect or not while Multi-label Classifier trained only with Images with defect. Multi-label Classifier have 4 output probability i.e. [defect_1,defect_2,defect_3,defect_4].

    Segmentation models are trained on each defect separately and predicts mask for each defect. Overall,we have 4 segment models i.e. one for each defect. There are total 6 models that are saved and later uses area threshold and class probability (different for each defect, depending on particular defect mask area and class probability).

    At last Encoded Pixels for test images are predicted and submitted. While training one must take care that proper data is fed to each model in the network which will help to reduce loss.



In [9]:
def f1_score(y_true, y_pred):
    #https://aakashgoel12.medium.com/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d
    #https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    true_positives=K.sum(K.round(K.clip(y_true*y_pred,0,1)))   #calculates number of true positives
    possible_positives=K.sum(K.round(K.clip(y_true,0,1)))      #calculates number of actual positives
    predicted_positives=K.sum(K.round(K.clip(y_pred,0,1)))

    #K.epsilon takes care of non-zero divisions
    #was modified by adding the constant epsilon, in order to avoid division by 0. Thus NaN will not be computed.
    precision=true_positives/(predicted_positives +K.epsilon())
    recall=true_positives/(possible_positives+K.epsilon())
    f1_val=2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [10]:
#https://stackoverflow.com/questions/31273652/how-to-calculate-dice-coefficient-for-measuring-accuracy-of-image-segmentation-i
def dice_coef(y_true,y_pred):
    y_true_f=tf.reshape(tf.dtypes.cast(y_true, tf.float32),[-1])
    y_pred_f=tf.reshape(tf.dtypes.cast(y_pred, tf.float32),[-1])
    intersection=tf.reduce_sum(y_true_f*y_pred_f)
    return (2. * intersection + 1.) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + 1.)

#https://stackoverflow.com/questions/49785133/keras-dice-coefficient-loss-function-is-negative-and-increasing-with-epochs
def dice_loss(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    return (1-dice_coefficient(y_true, y_pred))

#defining function for calculation of loss function: binary cross entropy + dice loss
def bce_dice_loss(y_true, y_pred):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    return binary_crossentropy(y_true, y_pred) + (1-dice_coef(y_true, y_pred))

In [11]:
def sum_pixel(i):
    return sum([int(k) for k in i.split(' ')[1::2]])

In [12]:
#Reference: https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [13]:
# Implementing custom data generator
#https://towardsdatascience.com/implementing-custom-data-generators-in-keras-de56f013581c
class test_DataGenerator(keras.utils.all_utils.Sequence):

  def __init__(self,dataframe,batch_size=1,shuffle=False,preprocess=None,info={}):
    self.batch_size = batch_size
    self.df = dataframe
    self.indices = self.df.index.tolist()
    self.preprocess = preprocess
    self.shuffle = shuffle
    self.on_epoch_end()

  def __len__(self):
    return len(self.indices) // (self.batch_size)

  def __getitem__(self, index):
    index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
    batch = [self.indices[k] for k in index]

    X= self.__get_data(batch)
    return X

  def on_epoch_end(self):
    self.index = np.arange(len(self.indices))
    if self.shuffle == True:
      np.random.shuffle(self.index)

  def __get_data(self, batch):
    X = np.empty((self.batch_size,256,800,3),dtype=np.float32) # image place-holders


    for i, id in enumerate(batch):
      X[i,] = Image.open('/content/steel/test_images/' + str(self.df['image_id'].loc[id])).resize((800,256))


      # preprocess input
    if self.preprocess!=None: X = self.preprocess(X)

    return X

In [14]:
binary=load_model('/content/binary_Xception_old.h5',custom_objects={'f1_score':f1_score})
multi=load_model('/content/multi_label.old.h5',custom_objects={'f1_score':f1_score})
segment_1=load_model('/content/segmentation_defect_1.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})
# segment_2=load_model('/content/drive/MyDrive/segmentation_defect_2.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})
# segment_3=load_model('/content/drive/MyDrive/segmentation_defect_3.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})
# segment_4=load_model('/content/drive/MyDrive/segmentation_defect_4.h5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

Loading all saved models.

In [15]:
test_folder_path='/content/steel/test_images'

In [16]:
def threshold(X):
  t=[]

  for i in range(len(X)):

    if sum_pixel(X['rle_1'].iloc[i])>=300 and sum_pixel(X['rle_1'].iloc[i])<=13500 and X['defect'].iloc[i]>=0.4 and X['defect_1'].iloc[i]>=0.5:
      t.append([X['image_id'].iloc[i]+'_1',X['rle_1'].iloc[i]])
    else:
      t.append([X['image_id'].iloc[i]+'_1',''])

    if sum_pixel(X['rle_2'].iloc[i])>=500 and sum_pixel(X['rle_2'].iloc[i])<=9000 and X['defect'].iloc[i]>=0.4 and X['defect_2'].iloc[i]>=0.5:
      t.append([X['image_id'].iloc[i]+'_2',X['rle_2'].iloc[i]])
    else:
      t.append([X['image_id'].iloc[i]+'_2',''])

    if sum_pixel(X['rle_3'].iloc[i])>=900 and sum_pixel(X['rle_3'].iloc[i])<=140000 and X['defect'].iloc[i]>=0.5 and X['defect_3'].iloc[i]>=0.6:
      t.append([X['image_id'].iloc[i]+'_3',X['rle_3'].iloc[i]])
    else:
      t.append([X['image_id'].iloc[i]+'_3',''])

    if sum_pixel(X['rle_4'].iloc[i])>=2400 and sum_pixel(X['rle_4'].iloc[i])<=120000 and X['defect'].iloc[i]>=0.4 and X['defect_4'].iloc[i]>=0.5:
      t.append([X['image_id'].iloc[i]+'_4',X['rle_4'].iloc[i]])
    else:
      t.append([X['image_id'].iloc[i]+'_4',''])

  df=pd.DataFrame(t,columns=['imageid_classid','rle'])
  return df

Depending on mask area(plot drawn in EDA) and class probability thresholds are decided for each defects respectively.

In [18]:
def predict(X):

   preprocess=get_preprocessing('efficientnetb5')
   datagen=ImageDataGenerator(rescale=1./255)
   data_generator=datagen.flow_from_dataframe(dataframe=X,
                                           directory=test_folder_path,
                                           x_col="image_id",
                                           class_mode=None,
                                           target_size=(256,512),
                                           batch_size=1,
                                           shuffle=False)
   a=[]

   pred_binary=binary.predict_generator(data_generator)
   pred_multi=multi.predict_generator(data_generator)
   classify =pd.DataFrame(pred_multi,columns=['defect_1','defect_2','defect_3','defect_4'])
   classify['defect']=pred_binary
   classify['image_id']=X['image_id']


   batch=test_DataGenerator(X,preprocess=preprocess)
   pred_1=segment_1.predict_generator(batch)
   pred_2=segment_2.predict_generator(batch)
   pred_3=segment_3.predict_generator(batch)
   pred_4=segment_4.predict_generator(batch)
   for i in range(len(pred_1)):
     v1=mask2rle(np.array((Image.fromarray((pred_1[i][:,:,0])>=0.5)).resize((1600,256))).astype(int))
     v2=mask2rle(np.array((Image.fromarray((pred_2[i][:,:,0])>=0.5)).resize((1600,256))).astype(int))
     v3=mask2rle(np.array((Image.fromarray((pred_3[i][:,:,0])>=0.5)).resize((1600,256))).astype(int))
     v4=mask2rle(np.array((Image.fromarray((pred_4[i][:,:,0])>=0.5)).resize((1600,256))).astype(int))
     a.append([X.image_id.iloc[i],v1,v2,v3,v4])
   segment=pd.DataFrame(a,columns=['image_id','rle_1','rle_2','rle_3','rle_4'])

   df=classify.merge(segment,on=['image_id'])

   df1=threshold(df)
   return df1


    First compute binary and multi-label classification later use segmentation model and after checking thresholds for each defect finally predicts Encoded pixels for each image id and class id .



# Prediction on Unseen given Test data


In [19]:
test=pd.DataFrame(test_image,columns=['image_id'])
test1=test[0:20]
test1=test1.reset_index().drop('index',axis=1)
data=predict(test1)
data

Found 20 validated image filenames.


NameError: name 'segment_2' is not defined

In [ ]:
a=[]
b=[]
for i in data.imageid_classid.values:
  k,l=i.split('_')
  a.append(k)
  b.append(l)

In [ ]:
df=pd.DataFrame(columns=['image_Id','class_Id','EncodedPixels'])
df['image_id']=a
df['class_id']=b
df['EncodedPixels']=data.rle.values
df.drop(['image_Id','class_Id'],axis=1,inplace=True)

In [ ]:
#https://www.analyticsvidhya.com/blog/2020/03/pivot-table-pandas-python/
df1=pd.pivot_table(df,values='EncodedPixels',index='image_id',columns='class_id',aggfunc=np.sum).astype(str)
df1=df1.reset_index()
df1.columns=['image_id','rle_1','rle_2','rle_3','rle_4']
df1

In [ ]:
def plot_mask(d):
  d=d.reset_index().drop('index',axis=1)
  test_folder_path='/content/drive//My Drive/Steel_Detection /test_images/'
  # Create figure and axes
  fig,ax=plt.subplots(d.shape[0],5,figsize=(17,8))
  for i in range(d.shape[0]):
    image_id=d['image_id'][i]
    rle_1=d['rle_1'][i]
    rle_2=d['rle_2'][i]
    rle_3=d['rle_3'][i]
    rle_4=d['rle_4'][i]
    im=Image.open(test_folder_path+str(image_id))
    ax[i,0].imshow(im)
    ax[i,0].set_title(image_id)

    mask=rle2mask(rle_1)
    ax[i,1].imshow(mask)
    ax[i,1].set_title("Defect_1")

    mask=rle2mask(rle_2)
    ax[i,2].imshow(mask)
    ax[i,2].set_title("Defect_2")

    mask=rle2mask(rle_3)
    ax[i,3].imshow(mask)
    ax[i,3].set_title("Defect_3")

    mask=rle2mask(rle_4)
    ax[i,4].imshow(mask)
    ax[i,4].set_title("Defect_4")

  fig.set_facecolor("tan")
  plt.show()


In [ ]:
def rle2mask(rle):
    # CONVERT RLE TO MASK
    if (pd.isnull(rle))|(rle=='')|(rle=='-1'):
        return np.zeros((256,1600) ,dtype=np.uint8)

    height= 256
    width = 1600
    mask= np.zeros( width*height ,dtype=np.uint8)

    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]-1
    lengths = array[1::2]
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1

    return mask.reshape( (height,width), order='F' )

In [ ]:
plot_mask(df1[0:5])

In [ ]:
plot_mask(df1[5:10])

In [ ]:
plot_mask(df1[10:15])

In [ ]:
plot_mask(df1[15:20])